FORMAT OF THE METHOD 

In [1]:
from numpy import array, argmin, inf
from fractions import Fraction

class Tableau:
    def __init__(self, obj):
        self.obj = [1] + obj
        self.rows = []
        self.cons = []
        self.no_variables = len(obj)
        self.no_constraints = 0
        self.is_fraction = False
    
    def add_constraint(self, expression, value):
        self.rows.append([0] + expression)
        self.cons.append(value)
        self.no_constraints += 1
    
    def _pivot_column(self):
        lowest_value = min(self.obj[1:-1])
        if lowest_value >= 0:
            return -1
        return argmin(self.obj[1:-1]) + 1
 
    def _pivot_row(self, col):
        rhs = [row[-1] for row in self.rows]
        lhs = [row[col] for row in self.rows]
        ratios = []
        for i in range(len(rhs)):
            if lhs[i] <= 0:
                ratios.append(inf)
            else:
                ratios.append(rhs[i] / lhs[i])
        return argmin(ratios)
 
    def _pivot(self, row, col):
        pivot_value = self.rows[row][col]
        self.rows[row] /= pivot_value
        for r in range(len(self.rows)):
            if r == row:
                continue
            self.rows[r] -= self.rows[r][col] * self.rows[row]
        self.obj -= self.obj[col] * self.rows[row]
 
    def _check(self):
        return min(self.obj[1:-1]) >= 0
         
    def solve(self):
        for i in range(len(self.rows)):
            self.obj += [0]
            ident = [0 for r in range(len(self.rows))]
            ident[i] = 1
            self.rows[i] += ident + [self.cons[i]]
            self.rows[i] = array(self.rows[i], dtype=float)
        self.obj = array(self.obj + [0], dtype=float)
 
        while not self._check():
            column_to_pivot = self._pivot_column()
            if column_to_pivot == -1:
                break
            row_to_pivot = self._pivot_row(column_to_pivot)
            self._pivot(row_to_pivot, column_to_pivot)
    
    def display(self):   
        fmt = '{:<8}'.format("Basic") \
              + "".join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)])   \
              + "".join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) \
              + '{:>8}'.format("Sol.")

        fmt += "\n" 
        fmt += '{:<8}'.format("z") + "".join(["{:>8.2f}".format(item) for item in self.obj[1:]])

        for i, row in enumerate(self.rows):
            fmt += "\n" 
            fmt += '{:<8}'.format(self.cons[i]) \
                   + "".join(["{:>8.2f}".format(item) for item in row[1:]])
        print(fmt)
        
        print("\nDual LP Problem:")
        for i, row in enumerate(self.rows):
            print(f"y{i+1} +", end=" ")
            for j, item in enumerate(row[1:-1]):
                if item != 0:
                    print(f"{abs(item)}*x{j+1}", end=" ")
            print(">= 1")

if __name__ == '__main__':
    t = Tableau([-1,-4,-2])
    t.add_constraint([5, 2, 2], 145)
    t.add_constraint([4, 8, -8], 260)
    t.add_constraint([1, 1, 4], 190)
    t.is_fraction = True
    t.solve()


Question 1 : writing the DUAL LP problem

In [2]:
from scipy.optimize import linprog

# Objective coefficients for the primal problem
c_primal = [1, 4, 2]  

# Constraint coefficients for the primal problem
A_primal = [
    [5, 2, 2],  # Coefficients of x1, x2, x3 in the first constraint
    [4, 8, -8],  # Coefficients of x1, x2, x3 in the second constraint
    [1, 1, 4]   # Coefficients of x1, x2, x3 in the third constraint
]

# Right-hand side values of the constraints for the primal problem
b_primal = [145, 260, 190]  

# Objective coefficients for the dual LP problem
c_dual = b_primal  

# Constraint coefficients for the dual LP problem
A_dual = [[A_primal[i][j] for i in range(len(A_primal))] for j in range(len(A_primal[0]))]

# Right-hand side values of the constraints for the dual LP problem
b_dual = c_primal  

print("Dual LP problem:")
print("Minimize:", c_dual)
print("Subject to:")
for i in range(len(A_dual)):
    print(A_dual[i], ">= ", b_dual[i])

C:\Users\jean-\anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Dual LP problem:
Minimize: [145, 260, 190]
Subject to:
[5, 4, 1] >=  1
[2, 8, 1] >=  4
[2, -8, 4] >=  2


Question 2 : Verification that (0,52.5,20) is a feasible solution for the primal

In [7]:
# Coefficients of the constraints for the primal problem
A_primal = [
    [5, 2, 2],    # Coefficients of x1, x2, x3 in the first constraint
    [4, 8, -8],   # Coefficients of x1, x2, x3 in the second constraint
    [1, 1, 4]     # Coefficients of x1, x2, x3 in the third constraint
]

# Right-hand side values of the constraints for the primal problem
b_primal = [145, 260, 190]

# Values of Q
Q = [0, 52.5, 20]

# Verification of feasibility of Q for the primal problem
print("Verification of feasibility of Q for the primal problem:")

for i in range(len(b_primal)):
    left_side = sum([A_primal[i][j] * Q[j] for j in range(len(Q))])  # Calculation of the left-hand side of the constraint
    print(f"Constraint {i+1}: {left_side} <= {b_primal[i]}")
    if left_side <= b_primal[i]:
        print("   This constraint is satisfied as the left-hand side is less than or equal to the right-hand side.")
    else:
        print("   This constraint is not satisfied as the left-hand side is greater than the right-hand side.")

# Displaying Q
print("\nQ =", Q)


Verification of feasibility of Q for the primal problem:
Constraint 1: 145.0 <= 145
   This constraint is satisfied as the left-hand side is less than or equal to the right-hand side.
Constraint 2: 260.0 <= 260
   This constraint is satisfied as the left-hand side is less than or equal to the right-hand side.
Constraint 3: 132.5 <= 190
   This constraint is satisfied as the left-hand side is less than or equal to the right-hand side.

Q = [0, 52.5, 20]


Question 3 : Use of CS to determine the candidate solution to the dual 

In [5]:
from ortools.linear_solver import pywraplp

def solve_lp(c, A, b):
    # Creating the solver
    solver = pywraplp.Solver.CreateSolver('GLOP')
    
    # Number of variables
    n = len(c)
    
    # Creating variables x
    x = [solver.NumVar(0, solver.infinity(), 'x{}'.format(i)) for i in range(n)]

    # Adding constraints
    for i in range(len(A)):
        constraint_expr = solver.RowConstraint(-solver.infinity(), b[i], '')
        for j in range(len(A[i])):
            constraint_expr.SetCoefficient(x[j], A[i][j])

    # Defining the objective
    objective = solver.Objective()
    for j in range(n):
        objective.SetCoefficient(x[j], c[j])
    objective.SetMaximization()

    # Solving the problem
    status = solver.Solve()

    # Displaying the solution
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        for i in range(n):
            print('x{} = {}'.format(i, x[i].solution_value()))
    else:
        print('The problem does not have an optimal solution.')

# Example
c = [1, 4, 2]
A = [[5, 2, 2], [4, 8, -8], [1, 1, 4]]
b = [145, 260, 190]

solve_lp(c, A, b)



Solution:
Objective value = 249.99999999999994
x0 = 0.0
x1 = 52.49999999999999
x2 = 19.999999999999993


Question 4 : We can observe that $Q$ is very close to the the feasible solution (0,52.5,20) which means that the $Q$ is the solution for the primal problem. 